In [2]:
import sys
sys.path.insert(0, '../')

from utils.data_utils import BratsDataset3D

In [3]:
scan_dir = r'/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
scan_dataset = BratsDataset3D(scan_dir, scan_type='flair')
print(scan_dataset.sorted_samples)
input_data = scan_dataset.__getitem__(0)
print(input_data[0].shape)
print(input_data[1].shape)


['/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_003', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_004', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_005', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_006', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_007', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_008', '/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_009', '/Users/colinodowd